In [3]:
import numpy as np
import pandas as pd
from scipy import stats

**Задание 1**  
Вернемся к набору данных о видеоиграх. Ответьте на следующие вопросы:  
1) Как критики относятся к спортивным играм?  
2) Критикам нравятся больше игры на PC или на PS4?  
3) Критикам больше нравятся стрелялки или стратегии?  

Для каждого вопроса:  
* сформулируйте нулевую и альтернативную гипотезы;
* выберите пороговый уровень статистической значимости;
* опишите полученные результаты статистического теста.

In [4]:
df = pd.read_csv('video_games_sales.csv')
df.head(3)

,Rank,Name,basename,Genre,ESRB_Rating,Platform,Publisher,Developer,VGChartz_Score,Critic_Score,...,NA_Sales,PAL_Sales,JP_Sales,Other_Sales,Year,Last_Update,url,status,Vgchartzscore,img_url
0,1,Wii Sports,wii-sports,Sports,E,Wii,Nintendo,Nintendo EAD,NaN,7.7,...,NaN,NaN,NaN,NaN,2006.0,NaN,http://www.vgchartz.com/game/2667/wii-sports/?...,1,NaN,/games/boxart/full_2258645AmericaFrontccc.jpg
1,2,Super Mario Bros.,super-mario-bros,Platform,NaN,NES,Nintendo,Nintendo EAD,NaN,10.0,...,NaN,NaN,NaN,NaN,1985.0,NaN,http://www.vgchartz.com/game/6455/super-mario-...,1,NaN,/games/boxart/8972270ccc.jpg
2,3,Mario Kart Wii,mario-kart-wii,Racing,E,Wii,Nintendo,Nintendo EAD,NaN,8.2,...,NaN,NaN,NaN,NaN,2008.0,11th Apr 18,http://www.vgchartz.com/game/6968/mario-kart-w...,1,8.7,/games/boxart/full_8932480AmericaFrontccc.jpg


In [5]:
df = df[['Genre', 'Platform', 'Critic_Score']]    # Оставим 3 нужных столбца
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55792 entries, 0 to 55791
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Genre         55792 non-null  object 
 1   Platform      55792 non-null  object 
 2   Critic_Score  6536 non-null   float64
dtypes: float64(1), object(2)
memory usage: 1.3+ MB


**Вопрос №1**  
Как критики относятся к спортивным играм?

H0: Критики относятся к спортивным играм примерно так же, как в среднем  
H1: Критики относятся к спортивным играм лучше (или хуже), чем в среднем  
Уровень значимости 0.05

In [6]:
a = 0.05    # Определим уровень значимости здесь и далее 0.05

In [7]:
mean_critic_score = df.Critic_Score.mean()    # Определим средний рейтинг от критиков по всем играм
mean_critic_score

7.21370869033052

In [8]:
df_sport = df[df.Genre == 'Sports'].Critic_Score.dropna()    # Выберем оценки для спортивных игр и удалим пропуски
df_sport.mean()

7.291424418604655

In [9]:
# Протестируем нулевую гипотезу, и если она не верна (т.е. pvalue < 0.05), посмотрим в какую сторону смещена статистика
stats.ttest_1samp(df_sport, mean_critic_score)

Ttest_1sampResult(statistic=1.385953985922894, pvalue=0.16621064929327967)

**Вывод 1:**  
по результатам теста нельзя отвергнуть нулевую гипотезу, т.е. критики относятся к спортивным играм примерно так же, как в среднем.
Хотя среднее по спортивным играм в данной выборке чуть больше, чем средний рейтинг по всем играм в данной выборке, но превышение не является статистически значимым.  

P.S. Фактически этот кейс аналогичен кейсам про Apple и Самсунг, которые были на лекции. Честно говоря, не вижу изъянов при моем способе формулирования гипотез и их проверки. Так что возникает вопрос про оправданность деления pvalue на 2, которое использовалась во время лекции.

**Вопрос №2**  
Критикам нравятся больше игры на PC или на PS4?  

H0: Критикам примерно одинаково нравятся игры на PC & PS4  
H1: Критикам нравятся игры на PC больше (или меньше), чем на PS4  
Уровень значимости 0.05

In [10]:
df_PC = df[df.Platform == 'PC'].Critic_Score.dropna()
df_PS4 = df[df.Platform == 'PS4'].Critic_Score.dropna()
print(df_PC.mean(), df_PS4.mean())

7.541849710982658 7.904587155963299


In [11]:
result = stats.ttest_ind(df_PC, df_PS4, equal_var=False)
print(result)
if result.pvalue < a:
    print("Отвергаем нулевую гипотезу")
    if result.statistic > 0:
        print("Критики относятся к играм на PC лучше, чем к играм на PS4")
    else:
        print("Критики относятся к играм на PS4 лучше, чем к играм на PC")
else:
    print("Не отвергаем нулевую гипотезу; критики относятся к играм на PC & PS4 примерно одинаково")

Ttest_indResult(statistic=-2.7394476056351627, pvalue=0.006931808250254211)
Отвергаем нулевую гипотезу
Критики относятся к играм на PS4 лучше, чем к играм на PC


**Вопрос №3**  
Критикам больше нравятся стрелялки или стратегии?

H0: Критикам примерно одинаково нравятся стрелялки и стратегии  
H1: Критикам нравятся стрелялки больше (или меньше), чем стратегии  
Уровень значимости 0.05

In [12]:
df_shooter = df[df.Genre == 'Shooter'].Critic_Score.dropna()
df_strategy = df[df.Genre == 'Strategy'].Critic_Score.dropna()
print(df_shooter.mean(), df_strategy.mean())

7.2868327402135264 7.429268292682925


In [13]:
result2 = stats.ttest_ind(df_shooter, df_strategy, equal_var=False)
print(result2)
if result2.pvalue < a:
    print("Отвергаем нулевую гипотезу")
    if result2.statistic > 0:
        print("Критики относятся к стрелялкам лучше, чем к стратегиям")
    else:
        print("Критики относятся к стратегиям лучше, чем к стрелялкам")
else:
    print("Не отвергаем нулевую гипотезу; критики относятся стрелялкам и стратегиям примерно одинаково")

Ttest_indResult(statistic=-1.6073949711166526, pvalue=0.10838786414223071)
Не отвергаем нулевую гипотезу; критики относятся стрелялкам и стратегиям примерно одинаково


**Задание 2**  
Реализуйте базовую модель логистической регрессии для классификации текстовых сообщений (используемые данные здесь) по признаку спама. Для этого:  
1) Приведите весь текст к нижнему регистру;  
2) Удалите мусорные символы;  
3) Удалите стоп-слова;  
4) Приведите все слова к нормальной форме;  
5) Преобразуйте все сообщения в вектора TF-IDF.  
Можете поэкспериментировать с параметрами TfidfVectorizer;  

6) Разделите данные на тестовые и тренировочные в соотношении 30/70, укажите random_state=42. Используйте train_test_split;  
7) Постройте модель логистической регрессии, укажите random_state=42, оцените ее точность на тестовых данных;  
8) Опишите результаты при помощи confusion_matrix;  
9) Постройте датафрейм, который будет содержать все исходные тексты сообщений, классифицированные неправильно (с указанием фактического и предсказанного).  

In [14]:
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [15]:
spam = pd.read_csv('spam.csv')
spam.head(3)

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...


In [16]:
spam['words'] = spam.Message.str.lower()    # Переводим в нижний регистр, создаём новую колонку
spam.head(3)

,Category,Message,words
0,ham,"Go until jurong point, crazy.. Available only ...","go until jurong point, crazy.. available only ..."
1,ham,Ok lar... Joking wif u oni...,ok lar... joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,free entry in 2 a wkly comp to win fa cup fina...


In [17]:
spam['words'] = spam.words.apply(lambda x: re.sub('[\W_]+',' ', x))    # Удаляем мусорные символы
spam.head()

,Category,Message,words
0,ham,"Go until jurong point, crazy.. Available only ...",go until jurong point crazy available only in ...
1,ham,Ok lar... Joking wif u oni...,ok lar joking wif u oni
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,free entry in 2 a wkly comp to win fa cup fina...
3,ham,U dun say so early hor... U c already then say...,u dun say so early hor u c already then say
4,ham,"Nah I don't think he goes to usf, he lives aro...",nah i don t think he goes to usf he lives arou...


In [18]:
spam['words'] = spam.words.apply(lambda x: x.split())    # Разделяем на слова
spam.head()

,Category,Message,words
0,ham,"Go until jurong point, crazy.. Available only ...","[go, until, jurong, point, crazy, available, o..."
1,ham,Ok lar... Joking wif u oni...,"[ok, lar, joking, wif, u, oni]"
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,"[free, entry, in, 2, a, wkly, comp, to, win, f..."
3,ham,U dun say so early hor... U c already then say...,"[u, dun, say, so, early, hor, u, c, already, t..."
4,ham,"Nah I don't think he goes to usf, he lives aro...","[nah, i, don, t, think, he, goes, to, usf, he,..."


In [19]:
stopwords_set = set(stopwords.words('english'))    # Формируем множество из стоп-слов

In [20]:
spam['words'] = spam.words.apply(lambda x: [word for word in x if word not in stopwords_set])    # Убираем стоп-слова
spam.head()

,Category,Message,words
0,ham,"Go until jurong point, crazy.. Available only ...","[go, jurong, point, crazy, available, bugis, n..."
1,ham,Ok lar... Joking wif u oni...,"[ok, lar, joking, wif, u, oni]"
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,"[free, entry, 2, wkly, comp, win, fa, cup, fin..."
3,ham,U dun say so early hor... U c already then say...,"[u, dun, say, early, hor, u, c, already, say]"
4,ham,"Nah I don't think he goes to usf, he lives aro...","[nah, think, goes, usf, lives, around, though]"


In [21]:
wordnet_lemmatizer = WordNetLemmatizer()
spam['words'] = spam.words.apply(lambda x: [wordnet_lemmatizer.lemmatize(word) for word in x])    # Лемматизируем
spam.head(3)

,Category,Message,words
0,ham,"Go until jurong point, crazy.. Available only ...","[go, jurong, point, crazy, available, bugis, n..."
1,ham,Ok lar... Joking wif u oni...,"[ok, lar, joking, wif, u, oni]"
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,"[free, entry, 2, wkly, comp, win, fa, cup, fin..."


In [22]:
spam['words'] = spam.words.apply(lambda x: ' '.join(x))    # Обратно в строки
spam.head(3)

,Category,Message,words
0,ham,"Go until jurong point, crazy.. Available only ...",go jurong point crazy available bugis n great ...
1,ham,Ok lar... Joking wif u oni...,ok lar joking wif u oni
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,free entry 2 wkly comp win fa cup final tkts 2...


In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [37]:
# Преобразуем в вектора TF-IDF
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(spam.words)
names = tfidf.get_feature_names()
tfidf_matrix = pd.DataFrame(tfidf_matrix.toarray(), columns=names)

tfidf_matrix[tfidf_matrix['crazy'] > 0]['crazy']    # Проверим, что матрица не пустая

0       0.271104
1011    0.314864
1344    0.230861
1668    0.455703
2803    0.347767
2848    0.418694
2930    0.268107
3099    0.630010
3752    0.397385
3942    0.308587
4752    0.220369
4974    0.269057
5098    0.221293
5511    0.541000
Name: crazy, dtype: float64

In [38]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [39]:
tfidf_matrix['Message'] = spam['Message']    # Добавим колонки категорий и исходных сообщений в TF-IDF матрицу перед раздением
tfidf_matrix['Category'] = spam['Category']
tfidf_matrix[['free', 'Message', 'Category']].head()    # Проверим, что получилось

,free,Message,Category
0,0.000000,"Go until jurong point, crazy.. Available only ...",ham
1,0.000000,Ok lar... Joking wif u oni...,ham
2,0.115769,Free entry in 2 a wkly comp to win FA Cup fina...,spam
3,0.000000,U dun say so early hor... U c already then say...,ham
4,0.000000,"Nah I don't think he goes to usf, he lives aro...",ham


In [40]:
data_train, data_test = train_test_split(tfidf_matrix, test_size=0.3, random_state=42)    # Разбиваем на 30/70

In [41]:
y_train = data_train['Category']
y_test = data_test[['Category', 'Message']]

del data_train['Category']
del data_test['Category']
del data_train['Message']
del data_test['Message']

In [42]:
clf = LogisticRegression(random_state=42).fit(data_train, y_train)

In [43]:
clf.predict(data_test)

array(['ham', 'ham', 'ham', ..., 'ham', 'spam', 'ham'], dtype=object)

In [44]:
result = pd.DataFrame([y_test['Category'].values, clf.predict(data_test), y_test['Message'].values]).T
result.columns = ['Initial_category', 'Predicted', 'Message']
result

,Initial_category,Predicted,Message
0,ham,ham,Squeeeeeze!! This is christmas hug.. If u lik ...
1,ham,ham,And also I've sorta blown him off a couple tim...
2,ham,ham,Mmm thats better now i got a roast down me! i...
3,ham,ham,Mm have some kanji dont eat anything heavy ok
4,ham,ham,So there's a ring that comes with the guys cos...
...,...,...,...
1667,ham,ham,"Hello, my boytoy! I made it home and my consta..."
1668,spam,spam,FREE entry into our £250 weekly comp just send...
1669,ham,ham,Aiyo u so poor thing... Then u dun wan 2 eat? ...
1670,spam,spam,"You have won ?1,000 cash or a ?2,000 prize! To..."


In [45]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test['Category'].values, clf.predict(data_test))
print(f"Confusion_matrix: \n{cm}")
print(f"Точность предсказания: {(cm[0][0] + cm[1][1]) / len(result):.1%}")

Confusion_matrix: 
[[1445    3]
 [  66  158]]
Точность предсказания: 95.9%


In [47]:
# Искомый датафрейм, который содержит все исходные тексты сообщений, классифицированные неправильно (с указанием фактического и предсказанного).
# Конечно, такие сообщения как "Are you free now?can i call now?" трактуются как спам (free-now-call)

final_df = result[result['Initial_category'] != result['Predicted']]
final_df

,Initial_category,Predicted,Message
17,ham,spam,Hey now am free you can call me.
40,spam,ham,Reminder: You have not downloaded the content ...
47,spam,ham,Guess what! Somebody you know secretly fancies...
74,spam,ham,Oh my god! I've found your number again! I'm s...
84,spam,ham,Your next amazing xxx PICSFREE1 video will be ...
...,...,...,...
1525,spam,ham,FreeMsg Hi baby wow just got a new cam moby. W...
1567,spam,ham,You have an important customer service announc...
1569,spam,ham,TheMob> Check out our newest selection of cont...
1576,ham,spam,Are you free now?can i call now?
